In [38]:
from dataclasses import dataclass

from typing import Dict, List, Tuple
Vector = List[float]
from dolang.vectorize import standard_function
from dolo.numeric.processes import Process, DiscretizedProcess
from dolo.compiler.objects import Domain, CartesianDomain
from dolo.numeric.grids import Grid, ProductGrid

from dolo.compiler.misc import CalibrationDict, calibration_to_vector
from numba import jit

import numpy as np

### Model Class definition

In [2]:
###
### Model API
###

# Here is a simple way to define a model
# which follows the model API

@dataclass
class PureModel:

    symbols: Dict[str, Vector]
    calibration: CalibrationDict
    functions: Dict[str, standard_function]
        
    # these are not accessed directly
    domain: Domain
    exogenous: Process
    
    def discretize(self, **kwargs)->Tuple[Grid, DiscretizedProcess]:
        
        # returns a discretized grid for exogenous and endogenous variables
        # and a discretization of the exogenous process
        
        # it can be inherited from dolo.numeric.model.Model

        dp = exogenous.discretize()
        endo_grid = self.domain.discretize()
        exo_grid = dp.grid
        grid = ProductGrid(exo_grid,endo_grid,names=['exo','endo'])
        return (grid, dp)

## Seb's redux

Simple model:

$$U(c) = \sqrt{c}$$
$$m_t = m_{t-1} + p - c$$
$$p \sim Normal(1,1)$$

### Symbols and Calibration in one dictionary

Define `symbol_calibration` and the functionalize conversion into model constructor arguments.

In [41]:
## Data Inputs

symbol_calibration = dict(
    states = {"m" : 0},
    controls = ["c"],
    exogenous = ["p"],
    parameters = {"β" : .99}
)

In [13]:
symbol_calibration

{'states': {'m': 0},
 'controls': ['c'],
 'exogenous': ['p'],
 'parameters': {'β': 0.99}}

In [42]:
def convert(symbol_calibration):
    symbols = {group : list(symbol_calibration[group].keys())
                   if type(symbol_calibration[group]) is dict
                   else symbol_calibration[group]
           for group
           in symbol_calibration}
    
    calibration_vector = {group : np.array(list(symbol_calibration[group].values()))
                   if type(symbol_calibration[group]) is dict
                   else np.array(symbol_calibration[group])
           for group
           in symbol_calibration}

    calibration = CalibrationDict(symbols, calibration_vector)
    
    return symbols, calibration

In [45]:
symbols, calibration = convert(symbol_calibration)

### Control domain and exogenous process

In a dictionary, set the exogenous process and the parameters of the domain object.

It may be possible to include these in the symbols_calibration dictionary defined above for compact representation.

In [60]:
## Dictionary for assigning domain and process values.
## Could it in principle be merged with the above dictionary,
## because calibration is not meaningful in those cases?

from dolo.numeric.distribution import UNormal

definitions = {
    'exogenous' : UNormal(μ = 1, σ=1),
    'controls' : {
        'c' : [0,10], ## the domain is here
    }
}

In [63]:
def make_model_definitions(definitions):
    
    domain = CartesianDomain(**definitions['controls'])
    
    return definitions['exogenous'], domain

In [65]:
exogenous, domain = make_model_definitions(definitions)

### Define model functions, and functionalize preprocessing into vectorized form?

In [71]:
import math
from numba import jit

# m, s, x, p = calibration['exogenous', 'states', 'controls', 'parameters']

# This is a convention in the code:
# it is `s` for $s_t$ and `S` for $s_{t+1}$.
# So g(s,S) is $g(s_t, s_{t+1})$ 
# We probably wrote it somewhere but I don't know where.
#

@jit(nopython=True)
def transition_(exo, states, controls, exo_plus,  params):
    
    m = states
    c = controls
    p = exo
    
    m_ = m - c + p

    return m_

@jit(nopython=True)
def felicity_(states, controls, params):
    
    m = states
    c = controls
    p = exo

    return math.sqrt(c)

In [81]:
from numba import guvectorize

def make_transition_gu(transition):

    @guvectorize(['void(float64[:], float64[:], float64[:], float64[:], float64[:], float64[:])'],
                 '(n_m),(n_s),(n_x),(n_m),(n_p)->(n_s)')
    def transition_gu(m,s,x,M,p,out):
        tmp = transition(m,s,x,M,p)
        for i in tmp:
            out[i] = tmp[i]
        
        return out
    
    return transition_gu

def make_felicity_gu(felicity):

    @guvectorize(['float64[:], float64[:], float64[:], float64[:]'],
                 '(n_s),(n_x),(n_p)->()') ## <-- Not sure output signature is correct !!
    def felicity_gu(s,x,p,out):
        tmp = felicity(s,x,p)
        for i in tmp:
            out[i] = tmp[i]
        
        return out
    
    return felicity

In [85]:
### TODO: a functionalize the mapping between the
### dictiony of JIT-ed functions passed in and the
### methods to gu_vectorize them, reducing model definition
### code

functions = {
    'transition' : make_transition_gu(transition_),
    'felicity' : make_felicity_gu(felicity_)
}

    
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "transition_gu" failed type inference due to: Invalid use of Function(<built-in function getitem>) with argument(s) of type(s): (array(float64, 1d, C), float64)
 * parameterized
In definition 0:
    All templates rejected with literals.
In definition 1:
    All templates rejected without literals.
In definition 2:
    All templates rejected with literals.
In definition 3:
    All templates rejected without literals.
In definition 4:
    All templates rejected with literals.
In definition 5:
    All templates rejected without literals.
In definition 6:
    All templates rejected with literals.
In definition 7:
    All templates rejected without literals.
In definition 8:
    All templates rejected with literals.
In definition 9:
    All templates rejected without literals.
In definition 10:
    All templates rejected with literals.
In definition 11:
    All templates rejected without literals.
I

### Build the model

In [86]:
model = PureModel(symbols,
                  calibration,
                  functions,
                  domain,
                  exogenous)


### Test

In [88]:
from dolo.algos.value_iteration import value_iteration
from dolo.algos.perturbation import perturb
from dolo.algos.simulations import simulate

#dr0 = perturb(model)

value_iteration(model)


#simulate(model, dr0)

KeyError: 'controls_lb'

## Pablo's Experiment

### Symbols

In [50]:
###
### Symbols definitions
###

# we define all variable names below
# to construct the calibration dictionary
# afterwards, it is only used to label outputs
# for instance in simulations


symbols = dict(
    states = ["z", "k"],
    controls = ["n", "i"],
    exogenous = ["e_z"],
    parameters = ["β", "δ", "α", "ρ", "σ", "η", "χ"]
)



In [51]:
###
### Calibration Definition
###
    
# ultimately, all is needed is a calibration object
# mapping variable names to values
# here we choose to rely on dolang methods
# to build this dictionary from the expressions
# of the steady-state/calibration

calibration_strings = dict(
    
    # parameters,
    β =  "0.99",
    δ =  "0.025",
    α =  "0.33",
    ρ =  "0.8",
    σ =  "5",
    η =  "1",

    zbar =  "0",
    χ =  "w/c**σ/n**η",
    c_i =  "1.5",
    c_y =  "0.5",
    e_z =  "0.0",

    # = "endogenous variables",
    n =  "0.33",
    z =  "zbar",
    rk =  "1/β-1+δ",
    w =  "(1-α)*exp(z)*(k/n)**(α)",
    k =  "n/(rk/α)**(1/(1-α))",
    y =  "exp(z)*k**α*n**(1-α)",
    i =  "δ*k",
    c =  "y - i",
    V =  "log(c)/(1-β)",
    u =  "c**(1-σ)/(1-σ) - χ*n**(1+η)/(1+η)",
    m =  "β/c**σ*(1-δ+rk)"
)

from dolang.triangular_solver import solve_triangular_system
calibration_dict = solve_triangular_system(calibration_strings)

calibration_vector = calibration_to_vector(symbols, calibration_dict)
calibration = CalibrationDict(symbols, calibration_vector)
calibration

In [52]:
calibration_vector

{'states': array([0.        , 9.35497829]),
 'controls': array([0.33      , 0.23387446]),
 'exogenous': array([0.]),
 'parameters': array([ 0.99      ,  0.025     ,  0.33      ,  0.8       ,  5.        ,
         1.        , 23.95785991])}

In [54]:
calibration['controls']

array([0.33      , 0.23387446])

### Functions

In [27]:
###
### Define functions
###

# the basic formulation of the functions (aka kernel)

# take and return "tuples" of floats because tuples are supposed 
# to be easy to be optimzed away by the compiler
# after the definition of the function, there is some boilerplate
# to translate these functions into broadcastable ones, with 
# optional numerical differentiation capabilities


# m, s, x, p = calibration['exogenous', 'states', 'controls', 'parameters']


#
# This is a convention in the code:
# it is `s` for $s_t$ and `S` for $s_{t+1}$.
# So g(s,S) is $g(s_t, s_{t+1})$ 
# We probably wrote it somewhere but I don't know where.
#

from numba import jit

# m, s, x, M+1, p -> s

@jit(nopython=True)
def transition_(m, s, x, M,  p):
    
    z, k = s
    n, i = x
    e_z, = m
    β, δ, α, ρ, σ, η, χ = p

    K = (1-δ)*k + i
    Z = (1-ρ)*z + e_z

    return (Z, K)

In [28]:
from math import exp

# m, s, x, M, S, X, p -> 

@jit(nopython=True)
def arbitrage_(m, s, x, M, S, X, p):

    z, k = s
    n, i = x
    e_z, = m
    β, δ, α, ρ, σ, η, χ = p

    Z, K = S
    N, I = X
    # E_z = M

    y = exp(z)*k**α*n**(1-α)
    c = y - i
    w = (1-α)*y/n


    Y = exp(Z)*K**α*N**(1-α)
    C = Y - I
    R = α*Y/K
    # W = (1-α)*y/n

    res_1 = χ*n**η*c**σ - w                    
    res_2 = 1 - β*(c/C)**(σ)*(1-δ+R) 

    ## Tupel of same size as X (controls)
    return (res_1, res_2)


In [29]:
m, s, x, p = calibration['exogenous', 'states', 'controls', 'parameters']

transition_(m,s,x,m,p) - s
arbitrage_(m,s,x,m,s,x,p)


(-4.440892098500626e-16, 0.0)

In [30]:

from numba import guvectorize

@guvectorize(['void(float64[:], float64[:], float64[:], float64[:], float64[:], float64[:])'],
             '(n_m),(n_s),(n_x),(n_m),(n_p)->(n_s)')
def transition_gu(m,s,x,M,p,out):
    tmp = transition_(m,s,x,M,p)
    out[0] = tmp[0]
    out[1] = tmp[1]
    return out


@guvectorize(['void(float64[:], float64[:], float64[:], float64[:], float64[:], float64[:], float64[:], float64[:])'],
             '(n_m),(n_s),(n_x),(n_m),(n_s),(n_x_),(n_p)->(n_s)')
def arbitrage_gu(m,s,x,M,S,X,p,out):
    tmp = arbitrage_(m,s,x,M,S,X,p)
    out[0] = tmp[0]
    out[1] = tmp[1]
    return out



    
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "transition_gu" failed type inference due to: No conversion from array(float64, 1d, A) to none for '$44return_value.18', defined at None

File "<ipython-input-30-fbbf3b05dd1a>", line 9:
def transition_gu(m,s,x,M,p,out):
    <source elided>
    out[1] = tmp[1]
    return out
    ^

[1] During: typing of assignment at <ipython-input-30-fbbf3b05dd1a> (9)

File "<ipython-input-30-fbbf3b05dd1a>", line 9:
def transition_gu(m,s,x,M,p,out):
    <source elided>
    out[1] = tmp[1]
    return out
    ^

    Function "transition_gu" was compiled in object mode without forceobj=True.

File "<ipython-input-30-fbbf3b05dd1a>", line 5:
             '(n_m),(n_s),(n_x),(n_m),(n_p)->(n_s)')
def transition_gu(m,s,x,M,p,out):
^

    
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit http://numba.pydata.org/nu

In [31]:
# now the functions transition_gu and arbitrage_gu
# are generelized ufuncs, which implement numpy-style
# broadcasting
transition_gu(m, s, x, m[None,:].repeat(10,axis=0), p)




array([[0.        , 9.35497829],
       [0.        , 9.35497829],
       [0.        , 9.35497829],
       [0.        , 9.35497829],
       [0.        , 9.35497829],
       [0.        , 9.35497829],
       [0.        , 9.35497829],
       [0.        , 9.35497829],
       [0.        , 9.35497829],
       [0.        , 9.35497829]])

In [32]:
# we use another convenience function to get a function which
# is able to compute derivatives
# I don't like this way to proceed, this should be rewritten
# currently it is only used for perturbations anyway.

from dolang.vectorize import standard_function
transition = standard_function(transition_gu, 2)
arbitrage = standard_function(arbitrage_gu, 2)
[g, g_m, g_s, g_x, g_m] = transition(m, s, x, m, p, diff=True)


functions = {
    'transition': transition,
    'arbitrage': arbitrage
}

In [33]:
[g, g_m, g_s, g_x, g_m]

[array([0.        , 9.35497829]),
 array([[0.],
        [0.]]),
 array([[0.2       , 0.        ],
        [0.        , 0.97500017]]),
 array([[0.        , 0.        ],
        [0.        , 0.99999991]]),
 array([[0.],
        [0.]])]

In [ ]:

###
### Exogenous process
###

from dolo.numeric.distribution import UNormal
exogenous = UNormal(σ=0.001)

# ###
# ### Discretized grid for endogenous states
# ###

# from dolo.numeric.grids import UniformCartesianGrid
# grid = UniformCartesianGrid(min=[-0.01, 5], max=[0.01, 15], n=[20, 20])

###
### Domain for endogenous states



domain = CartesianDomain(z=[-0.01, 0.01] , k=[5, 15])

In [40]:

###
### Solve model
###

# now we should be able to solve the model using
# any of the availables methods

model = PureModel(symbols,
                  calibration,
                  functions,
                  domain,
                  exogenous)


In [41]:
model

PureModel(symbols={'states': ['z', 'k'], 'controls': ['n', 'i'], 'exogenous': ['e_z'], 'parameters': ['β', 'δ', 'α', 'ρ', 'σ', 'η', 'χ']}, calibration=<dolo.compiler.misc.CalibrationDict object at 0x7f2b362b66d0>, functions={'transition': <dolang.vectorize.standard_function object at 0x7f2b35ab8490>, 'arbitrage': <dolang.vectorize.standard_function object at 0x7f2b35ab8990>}, domain={'z': array([-0.01,  0.01]), 'k': array([ 5., 15.])}, exogenous=<dolo.numeric.distribution.UNormal object at 0x7f2b3644b990>)

In [42]:
# now we should be able to solve the model using
# any of the availables methods

model = PureModel(symbols, calibration, functions, domain, exogenous)

from dolo.algos.time_iteration import time_iteration
from dolo.algos.perturbation import perturb
from dolo.algos.simulations import simulate
from dolo.algos.improved_time_iteration import improved_time_iteration

dr0 = perturb(model)

time_iteration(model)


simulate(model, dr0)


------------------------------------------------
| N   |  Error     | Gain     | Time     | nit |
------------------------------------------------
|   1 |  4.781e-01 |      nan |    5.060 |   5 |
|   2 |  1.131e-01 |    0.237 |    0.119 |   3 |
|   3 |  6.271e-02 |    0.554 |    0.144 |   3 |
|   4 |  4.381e-02 |    0.699 |    0.146 |   3 |
|   5 |  3.346e-02 |    0.764 |    0.143 |   3 |
|   6 |  2.654e-02 |    0.793 |    0.156 |   3 |
|   7 |  2.154e-02 |    0.812 |    0.103 |   2 |
|   8 |  1.779e-02 |    0.826 |    0.114 |   2 |
|   9 |  1.488e-02 |    0.837 |    0.115 |   2 |
|  10 |  1.261e-02 |    0.847 |    0.102 |   2 |
|  11 |  1.143e-02 |    0.907 |    0.100 |   2 |
|  12 |  1.038e-02 |    0.908 |    0.122 |   2 |
|  13 |  9.436e-03 |    0.909 |    0.141 |   2 |
|  14 |  8.591e-03 |    0.910 |    0.106 |   2 |
|  15 |  7.832e-03 |    0.912 |    0.088 |   2 |
|  16 |  7.150e-03 |    0.913 |    0.091 |   2 |
|  17 |  6.535e-03 |    0.914 |    0.095 |   2 |
|  18 |  5.981e-03 |

<xarray.DataArray (T: 40, N: 1, V: 4)>
array([[[ 0.00000000e+00,  9.35497829e+00,  3.30000000e-01,
          2.33874457e-01]],

       [[ 0.00000000e+00,  9.35497829e+00,  3.30000000e-01,
          2.33874457e-01]],

       [[ 1.61894637e-03,  9.35497829e+00,  3.30336545e-01,
          2.36125342e-01]],

       [[ 9.60004283e-04,  9.35722918e+00,  3.30165169e-01,
          2.35185517e-01]],

       [[ 8.47768511e-04,  9.35848396e+00,  3.30122663e-01,
          2.35016274e-01]],

       [[-6.67530972e-04,  9.35953814e+00,  3.29791555e-01,
          2.32898407e-01]],

       [[ 6.61336360e-04,  9.35844809e+00,  3.30084456e-01,
          2.34757448e-01]],

       [[-2.06444337e-04,  9.35924434e+00,  3.29891895e-01,
          2.33542564e-01]],

       [[ 1.29009400e-03,  9.35880579e+00,  3.30209695e-01,
          2.35627872e-01]],

       [[ 2.04715008e-03,  9.36046352e+00,  3.30341739e-01,
          2.36663003e-01]],

       [[-4.88524107e-04,  9.36311493e+00,  3.29774109e-01,
          2.33109671e-01]],

       [[-1.14538699e-03,  9.36214673e+00,  3.29652357e-01,
          2.32206591e-01]],

       [[-3.71070284e-04,  9.36029965e+00,  3.29841546e-01,
          2.33302579e-01]],

       [[ 1.00726593e-03,  9.35959474e+00,  3.30138845e-01,
          2.35226348e-01]],

       [[ 1.99689307e-04,  9.36083122e+00,  3.29952072e-01,
          2.34090538e-01]],

       [[ 4.08560977e-04,  9.36090098e+00,  3.29994426e-01,
          2.34380207e-01]],

       [[-9.76483411e-04,  9.36125866e+00,  3.29701039e-01,
          2.32450764e-01]],

       [[ 5.46886323e-04,  9.35967796e+00,  3.30041870e-01,
          2.34585389e-01]],

       [[ 1.17859834e-03,  9.36027140e+00,  3.30164121e-01,
          2.35457442e-01]],

       [[ 4.55236916e-04,  9.36172206e+00,  3.29991582e-01,
          2.34436467e-01]],

       [[-4.05912590e-05,  9.36211547e+00,  3.29882498e-01,
          2.33742961e-01]],

       [[ 6.72424374e-05,  9.36180555e+00,  3.29909651e-01,
          2.33896145e-01]],

       [[-3.01423549e-05,  9.36165655e+00,  3.29891683e-01,
          2.33762314e-01]],

       [[-5.93038132e-04,  9.36137745e+00,  3.29778934e-01,
          2.32982634e-01]],

       [[ 2.20801708e-04,  9.36032565e+00,  3.29964187e-01,
          2.34125209e-01]],

       [[-1.05387693e-03,  9.36044272e+00,  3.29697419e-01,
          2.32351742e-01]],

       [[-6.47608337e-04,  9.35878339e+00,  3.29807230e-01,
          2.32934044e-01]],

       [[-1.55620741e-03,  9.35774785e+00,  3.29634175e-01,
          2.31681674e-01]],

       [[-7.97541640e-04,  9.35548583e+00,  3.29826452e-01,
          2.32760266e-01]],

       [[ 1.94974293e-04,  9.35435895e+00,  3.30049995e-01,
          2.34152051e-01]],

       [[-1.17151170e-03,  9.35465203e+00,  3.29761453e-01,
          2.32249090e-01]],

       [[-5.92754716e-04,  9.35303482e+00,  3.29906477e-01,
          2.33070766e-01]],

       [[-6.64036048e-04,  9.35227971e+00,  3.29903198e-01,
          2.32979602e-01]],

       [[-3.99936958e-04,  9.35145232e+00,  3.29970742e-01,
          2.33355491e-01]],

       [[-2.39045464e-03,  9.35102150e+00,  3.29563539e-01,
          2.30592527e-01]],

       [[-1.37951489e-03,  9.34783849e+00,  3.29822332e-01,
          2.32031552e-01]],

       [[-3.63067194e-04,  9.34617408e+00,  3.30059064e-01,
          2.33462264e-01]],

       [[-2.34863019e-03,  9.34598199e+00,  3.29649242e-01,
          2.30703677e-01]],

       [[-1.29015454e-03,  9.34303612e+00,  3.29914293e-01,
          2.32206299e-01]],

       [[ 1.81730741e-05,  9.34166652e+00,  3.30207196e-01,
          2.34039723e-01]]])
Coordinates:
  * T        (T) int64 0 1 2 3 4 5 6 7 8 9 10 ... 30 31 32 33 34 35 36 37 38 39
  * N        (N) int64 0
  * V        (V) <U1 'z' 'k' 'n' 'i'